In [7]:
# import environment variables
from dotenv import load_dotenv
import os 

load_dotenv(override=True)

True

In [11]:
ZHIPU_API_KEY = os.getenv("ZHIPU_API_KEY")
# print(ZHIPU_API_KEY)

In [13]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model = 'glm-4-flash',
    openai_api_base = "https://open.bigmodel.cn/api/paas/v4/",
    openai_api_key = ZHIPU_API_KEY,
    temperature = 0.1,
)